## Define

In [1]:
from resources.utils import *
from gensim.models import Word2Vec

!mkdir -p tb
!mkdir -p tb/embeddings

mc['github'].list_collection_names()

['users_followers_embeddings',
 'followers',
 'users_following_embeddings',
 'followers_all',
 'users_watches',
 'repos_watches_embeddings',
 'users',
 'projects',
 'repos_stars_embeddings',
 'following']

## Repos

In [2]:
repo_model_name = 'repos_watches_emb_sg_d300_w50_mc200_e50'
repo_model = Word2Vec.load('./models/' + repo_model_name + '.bin')
print(repo_model)

Word2Vec(vocab=84973, size=300, alpha=0.025)


In [22]:
good_rids = []

for rid, r in repo_model.wv.vocab.items():
    if r.count >= 10_000:
        good_rids.append(int(rid))
len(good_rids)

974

In [13]:
def dropCategories(df, col, threshold):
    drop_list = df[col].value_counts()[threshold:].index.to_list()
    df[col].cat.remove_categories(drop_list, inplace=True)
    return df

In [23]:
repos_cur = mc['github']['projects'].find({"_id": {"$in": good_rids}}, ['url', 'language', 'created_at', 'updated_at'])
repos_emb_cur = mc['github']['repos_stars_embeddings'].find({"_id": {"$in": good_rids}})

repos_df = pd.DataFrame(list(repos_cur)).set_index('_id')[['url', 'language', 'created_at', 'updated_at']]
repos_df.language = repos_df.language.astype("category")
repos_df = dropCategories(repos_df, "language", 42)

repos_emb_df = pd.DataFrame(list(repos_emb_cur)).set_index('_id')
repos_emb_df.columns = ['n_vecs', 'embeddings']
repos_df = repos_df.join(repos_emb_df)
repos_df

,url,language,created_at,updated_at,n_vecs,embeddings
_id,,,,,,
6,cocos2d/cocos2d-x,C++,2010-11-18 23:17:00,2019-02-26 20:51:59,13901,"[0.24193669855594635, 0.1203954815864563, 0.49..."
37,angular/angular.js,JavaScript,2010-01-06 00:34:37,2019-02-27 03:14:26,56025,"[0.23754794895648956, -0.08706655353307724, 0...."
100,angular/angular-seed,JavaScript,2010-12-24 06:07:50,2019-02-26 05:55:12,14461,"[0.16231873631477356, -0.2859654724597931, 0.4..."
114,madrobby/zepto,HTML,2010-09-20 07:57:57,2019-02-26 04:14:17,15462,"[0.24657486379146576, -0.1155390813946724, 0.6..."
284,mongodb/mongo,C++,2009-01-15 16:15:18,2019-02-26 15:43:42,15866,"[0.34626296162605286, -0.06895570456981659, -0..."
334,hakimel/reveal.js,JavaScript,2011-06-07 18:54:22,2019-02-27 08:27:49,37126,"[0.27886509895324707, -0.10664886236190796, -0..."
337,documentcloud/backbone,JavaScript,2010-09-30 17:41:28,1970-01-01 01:00:01,12212,"[0.26180288195610046, -0.3434514105319977, -0...."
340,netty/netty,Java,2010-11-09 09:22:21,2019-02-27 03:35:28,18231,"[0.24259130656719208, -0.18594089150428772, 0...."
390,kennethreitz/requests,Python,2011-02-13 18:38:17,2019-02-26 13:59:13,25221,"[0.19590069353580475, 2.5026027287822217e-05, ..."


In [24]:
repos_df.language.value_counts()

JavaScript          284
Python               68
Java                 68
Go                   48
C++                  38
Objective-C          26
C                    25
Swift                24
Ruby                 24
HTML                 23
PHP                  20
CSS                  17
Shell                16
TypeScript           14
CoffeeScript          9
VimL                  8
C#                    7
Jupyter Notebook      6
Scala                 4
Lua                   4
Rust                  4
Assembly              3
Haskell               3
Clojure               2
TeX                   2
Objective-C++         2
Elixir                2
Dart                  1
Batchfile             1
XSLT                  1
Emacs Lisp            1
Groff                 1
Julia                 1
Vue                   1
Matlab                1
OCaml                 1
PowerShell            1
PureBasic             1
QML                   1
Kotlin                1
Name: language, dtype: int64

In [25]:
repos_df.to_pickle("./tb/embeddings/repos_1k_gte10k.pkl")

## Followers

In [199]:
followers_model_name = 'users_followers_emb_sg_d300_w30_mc3_e5'
followers_model = Word2Vec.load('./models/' + followers_model_name + '.bin')
print(followers_model)

Word2Vec(vocab=1446832, size=300, alpha=0.025)


In [222]:
followers_good_uids = []

for uid, r in model.wv.vocab.items():
    if r.count >= 100:
        followers_good_uids.append(int(uid))
len(followers_good_uids)

24435

In [223]:
followers_users_cur = mc['github']['users'].find({"_id": {"$in": followers_good_uids}}, 
                                       ['login', 'type', 'fake', 'location', 'deleted', 'country_code', 'created_at'])
followers_users_emb_cur = mc['github']['users_followers_embeddings'].find({"_id": {"$in": followers_good_uids}})

followers_users_df = pd.DataFrame(list(followers_users_cur)).set_index('_id')[
    ['login', 'type', 'fake', 'location', 'deleted', 'country_code', 'created_at']
]
followers_users_emb_df = pd.DataFrame(list(followers_users_emb_cur)).set_index('_id')
followers_users_emb_df.columns = ['n_vecs', 'embeddings']
followers_users_df = followers_users_df.join(followers_users_emb_df)
followers_users_df

,login,type,fake,location,deleted,country_code,created_at,n_vecs,embeddings
_id,,,,,,,,,
2,jmettraux,USR,False,Hiroshima,False,jp,2008-03-22 00:37:42,430,"[0.2063964605331421, 0.3219830095767975, 0.054..."
4,kennethkalmer,USR,False,"Johannesburg, South Africa",False,za,2008-04-28 17:25:53,102,"[0.2238793522119522, 0.2812013626098633, 0.088..."
7,hasimo,USR,False,Tokyo/Japan,False,jp,2008-09-30 00:05:24,256,"[0.3436596989631653, 0.6199936866760254, -0.04..."
10,mcollina,USR,False,In the clouds above Italy,False,None,2009-02-05 21:24:19,371,"[0.32903146743774414, 0.4905785024166107, 0.22..."
47,l4u,USR,False,Hong Kong,False,hk,2009-06-22 14:13:10,309,"[0.3731025159358978, 0.5073902010917664, 0.017..."
49,corefan,USR,False,None,False,None,2012-03-18 01:43:30,386,"[0.33945879340171814, -0.07949918508529663, 0...."
58,hcilab,ORG,False,"Fredericton, New Brunswick, Canada",False,ca,2012-04-04 03:12:28,21738,"[0.38965266942977905, 0.8585405349731445, 0.42..."
70,rafacm,USR,False,Vienna,False,at,2010-07-22 03:58:01,129,"[0.28958895802497864, 0.16287967562675476, 0.1..."
109,jfirebaugh,USR,False,"San Francisco, CA",False,us,2009-06-24 18:01:39,127,"[0.18331538140773773, 0.30185845494270325, 0.1..."


In [224]:
followers_users_df.to_pickle("./tb/embeddings/users_followers_24k_gte100.pkl")

## Following

In [225]:
following_model_name = 'users_following_emb_sg_d300_w30_mc3_e5'
following_model = Word2Vec.load('./models/' + following_model_name + '.bin')
print(following_model)

Word2Vec(vocab=1385758, size=300, alpha=0.025)


In [244]:
following_good_uids = []

for uid, r in following_model.wv.vocab.items():
    if r.count >= 1_000:
        following_good_uids.append(int(uid))
len(following_good_uids)

1537

In [245]:
following_users_cur = mc['github']['users'].find({"_id": {"$in": following_good_uids}}, 
                                       ['login', 'type', 'fake', 'location', 'deleted', 'country_code', 'created_at'])
following_users_emb_cur = mc['github']['users_following_embeddings'].find({"_id": {"$in": following_good_uids}})

following_users_df = pd.DataFrame(list(following_users_cur)).set_index('_id')[
    ['login', 'type', 'fake', 'location', 'deleted', 'country_code', 'created_at']
]
following_users_emb_df = pd.DataFrame(list(following_users_emb_cur)).set_index('_id')
following_users_emb_df.columns = ['n_vecs', 'embeddings']
following_users_df = following_users_df.join(following_users_emb_df)
following_users_df

,login,type,fake,location,deleted,country_code,created_at,n_vecs,embeddings
_id,,,,,,,,,
10,mcollina,USR,False,In the clouds above Italy,False,None,2009-02-05 21:24:19,2574,"[-0.13591870665550232, 0.30812159180641174, -0..."
27,jimweirich,USR,False,Cincinnati,False,us,2008-03-27 18:19:22,1905,"[-0.5672861933708191, 0.192423477768898, -0.17..."
78,samuelclay,USR,False,"Cambridge, Massachusetts",False,us,2009-01-05 03:46:08,1295,"[-0.44099417328834534, 0.1900487095117569, -0...."
129,tenderlove,USR,False,Seattle,False,us,2008-03-14 19:04:17,7904,"[-0.43326085805892944, 0.38780757784843445, -0..."
189,hmason,USR,False,"New York, NY",False,us,2008-07-22 15:55:09,3413,"[-0.27972522377967834, 0.024197496473789215, -..."
216,fredwu,USR,False,"Melbourne, Australia (& Shanghai, China)",False,None,2008-10-31 03:34:48,1244,"[-0.7697437405586243, -0.10295889526605606, -0..."
272,yui,ORG,False,"Sunnyvale, CA",False,us,2008-12-04 00:46:11,1270,"[-0.8946189284324646, -0.1585889607667923, -0...."
303,Marak,USR,False,"New York City, New York",False,us,2009-04-03 01:34:57,2436,"[-0.42212310433387756, 0.15440842509269714, -0..."
376,tianyicui,USR,False,New York City,False,us,2009-02-09 13:50:20,1614,"[-0.5902652144432068, 0.044978946447372437, -0..."


In [246]:
following_users_df.to_pickle("./tb/embeddings/users_following_2k_gte1k.pkl")

## sandbox

In [1]:
fls = !ls ./tb/embeddings/

for f in fls:
    tmp_df = pd.read_pickle(f'./tb/embeddings/{f}')
    print(f'{f} - {len(tmp_df)} - gte{tmp_df["n_vecs"].min()}')

repos_19k_gte1k.pkl - 18670 - gte1000
repos_1k_gte10k.pkl - 970 - gte10002
repos_85k_gte200.pkl - 84806 - gte200
users_followers_24k_gte100.pkl - 24435 - gte100
users_followers_70k_gte50.pkl - 69570 - gte50
users_following_29k_gte100.pkl - 28827 - gte100
users_following_2k_gte1k.pkl - 1537 - gte1000
users_following_88k_gte40.pkl - 88323 - gte40
